# TE_Connect Sales Forcasting

Author: Alexiy Buynitsky

### Configuration

Run Through the following commands, and then alter the:
- `src/data_constants.py`
  -  Control data inputs,outputs, prediction, foracsting...
- `src/model_constants.py`
  - Control model architecture, lstm cells, hidden layers, dropout...
- `src/filepath_constants.py`
  - Control file system, verbose mode...

You then only have to run the last command and use the [MLflow UI](https://dagshub.com/Abuynits/TE_Connect.mlflow) to track your experiments!



## Imports

In [ ]:
!pip3 install -r requirements.txt &> /dev/null
!pip3 install pytorch_lightning &> /dev/null
# install all libraries
!pip install dvc &> /dev/null
!pip install dagshub &> /dev/null
!pip3 install mlflow &> /dev/null

In [ ]:
import requests
import getpass
import datetime

In [ ]:
#@title Set Environment Variables

#@markdown Enter the repository name for the project:
REPO_NAME= "TE_Connect" #@param {type:"string"}

#@markdown Enter the username of your DAGsHub account:
USER_NAME = "Abuynits" #@param {type:"string"}

#@markdown Enter the email for your DAGsHub account:
EMAIL = "abuynits@gmail.com" #@param {type:"string"}

#@markdown ---

In [ ]:
# create a personal access token
r = requests.post('https://dagshub.com/api/v1/user/tokens', 
                  json={"name": f"colab-token-{datetime.datetime.now()}"}, 
                  auth=(USER_NAME, getpass.getpass('DAGsHub password:')))
r.raise_for_status()
TOKEN=r.json()['sha1']

## Generating SSH Key for github

In [ ]:
!ssh-keygen -t rsa -b 4096

hit enter for all to use default file location. 

**NOTE**: will be deleted at the end of the notebook session

In [ ]:
!ssh-keyscan -t rsa github.com >> ~/.ssh/known_hosts

Print the ssh_id and add it to github keys. Will only be valid for this notebook session

In [ ]:
!cat /root/.ssh/id_rsa.pub

In [ ]:
!ssh -T git@github.com # test the ssh id  

In [ ]:
# configure github usernames
!git config --global user.email {REPO_NAME} 
!git config --global user.name {USER_NAME}

In [ ]:
!git clone git@github.com:Abuynits/TE_Connect.git

In [ ]:
%cd TE_Connect

## Initialize DVC!

In [ ]:
!dvc remote modify origin --local auth basic 
!dvc remote modify origin --local user {USER_NAME} 
!dvc remote modify origin --local password {TOKEN}

In [ ]:
!dvc pull --remote origin

#TESTING

run the following cell after alterning any of the constant files. Track the progress live in the MLflow UI.

In [6]:
!MLFLOW_TRACKING_URI=https://dagshub.com/Abuynits/TE_Connect.mlflow \
MLFLOW_TRACKING_USERNAME={USER_NAME} \
MLFLOW_TRACKING_PASSWORD={TOKEN}  \
dvc repro

ERROR: you are not inside of a DVC repository (checked up to mount point '/')
